In [1]:
import _dunsros as DR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from kan import *
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# условно константы
k_og = 178                      #газовый фактор м.**3/ст.м.**3
bo = 1.197                      #объемный коэффициент нефти
bw = 1                          #объемный коэффициент воды
bg = 9.1 * 1e-3                 #объемный коэффициент газа
rhow = 1 * 1e3                  #плотность воды


# условно переменные
T = 10                          #температура гр.Ц.
d = 0.1524                      #диаметр трубы
theta = 90                      #угол наклона скважины
q0 = 1.59 * 1e3                 #дебит нефти н.у.
qg0 = 283 * 1e3                 #дебит газа н.у.
rs = 50.6                       #коэффициент растворимости газа в нефти (конденсации) или коэффициент выделения газа из нефти (выкипания)
rho0 = 8 * 1e2                  #плотность нефти
rhog = 7 * 1e2                  #плотность газа
mug = 0.016 * 1e-3              #вязкость газа
muo = 0.97 * 1e-3               #вязкость нефти
epsilon = 18.288 * 1e-6         #шероховатость стенки трубы
sigmao = 8.41 * 1e-3            #коэффициент поверхностного натяжения нефти
wc = 1 - qg0 / q0 / k_og        #коэффициент обводненности
p = 180                         #давление (где?)



# расчетные величины (кроме обводненности, которую удобно использовать как входную переменную)
qo = q0 * (1 - wc) * bo                                 #объем нефти в пластовых условиях
ql = ((1 - wc) * bo + wc * bw) * qo                     #объем жидкости в пластовых условиях
qg = (qg0 - qo * rs * (1 - wc)) * bg                    #объем газа в пластовых условиях
rho_lrc = rho0 * (1 - wc) / bo + rhow * wc / bw         #плотность жидкой фазы
rho_grc = rhog / rs                                     #плотность газа
                                                        #коэффициент поверхностного натяжения жидкость-газ
                                                        #вязкость жидкости
Sd = np.pi * d**2 / 4                                   #площадь сечения трубы

sec_p_day = 24 * 60 * 60                                #переведем все в систему СИ
qo = qo / sec_p_day
ql = ql / sec_p_day
qg = qg / sec_p_day

args_params = {
        "d": d,                         #диаметр в метрах
        "theta_deg": theta,             #угол
        "ql_rc_m3day": ql,              #дебит жидкости
        "qg_rc_m3day": qg,              #расход газа
        "rho_lrc_kgm3": rho_lrc,        #плотность жидкой фазы
        "rho_grc_kgm3": rho_grc,        #плотность газа
        "p": p,                         #давление (где?)
        "T" : T,                        #температура смеси
        "wc": wc,                       #заводнение
        "sigmao": sigmao,               #коэффициент поверхностного натяжения нефти
        "muo": muo,                     #вязкость нефти
}

calibr_C_grav = 1                       #коэффициент калибровки гравитационной компоненты градиента
calibr_C_fric = 1                       #коэффициент калибровки фрикционной компоненты градиента

args_grav_grad = {
        "theta_deg": theta,
        "c_calibr_grav": calibr_C_grav,
}

args_fric_grad = {
        "eps_m": epsilon,
        "mug_rc_cp": mug,
        "c_calibr_fric": calibr_C_fric,
        "rho_lrc_kgm3": rho_lrc,
        "rho_grc_kgm3": rho_grc,
        "T" : T,
        "wc": wc,
        "sigmao": sigmao,
}

In [3]:
# протестируем работоспособность класса DunsRos()
dr_corr = DR.DunsRos()
dr_corr.calc_params(**args_params)  #класс подразумевает предварительный рассчет параметров данным методом, иначе работать не будет
dr_corr.calc_grav(**args_grav_grad) + dr_corr.calc_fric(**args_fric_grad), dr_corr.calc_grav(**args_grav_grad), dr_corr.calc_fric(**args_fric_grad)

NameError: name 'vsm' is not defined

In [ ]:
# создадим функцию для вычисления градиента от аргументов (для более привычной записи)
# PD_DZ = lambda args_grav_grad, args_fric_grad: dr_corr.calc_grav(**args_grav_grad) + dr_corr.calc_fric(**args_fric_grad)
# list(args_fric_grad.values())

In [ ]:
count = 5
# Набираем диапазоны выбранных переменных для перебора. Диапазоны по длинне count короткие, т.к. при переборе будет n**count значений 
d = torch.tensor(np.linspace(0.11,0.32,count))                     #диаметры труб
theta = torch.tensor(np.linspace(0,90,count))                      #уголы наклона
q0 = torch.tensor(np.linspace(20,1000,count))                      #дебиты нефти н.у. м**3/сут.
qg0 = 283 * 1e3                 #дебит газа н.у.
rs = 50.6                       #коэффициент растворимости газа в нефти (конденсации) или коэффициент выделения газа из нефти (выкипания)
rho0 = 8 * 1e2                  #плотность нефти
rhog = 7 * 1e2                  #плотность газа
mug = 0.016 * 1e-3              #вязкость газа
muo = 0.97 * 1e-3               #вязкость нефти
epsilon = 18.288 * 1e-6         #шероховатость стенки трубы
sigmao = 8.41 * 1e-3            #коэффициент поверхностного натяжения нефти
wc = torch.tensor(np.linspace(0.1,0.98,count))         #коэффициент обводненности
p = torch.tensor(np.linspace(10,80,count))                         #давление (где?)



In [30]:
wc = torch.tensor(np.linspace(0.1,0.98,count))
wc

tensor([0.1000, 0.3200, 0.5400, 0.7600, 0.9800], dtype=torch.float64)

In [13]:
# let's construct a dataset
f = lambda x: x[:,0]**2 + 0.3*x[:,1] + 0.1*x[:,2]**3 + 0.0*x[:,3]
dataset = create_dataset(f, n_var=4, device=device)

input_vars = [r'$x_'+str(i)+'$' for i in range(4)]

model = KAN(width=[4,5,1], device=device)
model.fit(dataset, steps=40, lamb=0.001);

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 4.47e-03 | test_loss: 4.23e-03 | reg: 4.17e+00 | : 100%|█| 40/40 [00:09<00:00,  4.34it

saving model version 0.1


In [22]:
d = torch.tensor(np.linspace(0.12,0.2,5))
d

tensor([0.1200, 0.1400, 0.1600, 0.1800, 0.2000], dtype=torch.float64)